In [ ]:
%pip install ipympl -q
%pip install numpy -q
%pip install wheel

%pip install keras -q
%pip install keras-cv -q

%pip install matplotlib -q

In [ ]:
location_to_saved_model: str = '/home/spectre/environment_model.keras'

In [ ]:
import tensorflow

gpus = tensorflow.config.list_physical_devices(
    'GPU'
)

In [ ]:
nvidia_maximum_limit_for_gpu_memory: int = 5925

if gpus:
    try:
        for gpu in gpus:
            tensorflow.config.set_logical_device_configuration(
                gpu, 
                [
                    tensorflow.config.LogicalDeviceConfiguration(
                        memory_limit=nvidia_maximum_limit_for_gpu_memory
                    )
                ]
            )
    except RuntimeError as e:
        print(e)

location_to_dataset: str = '/tmp/dataset'
name_of_location: str = 'LocationDataset'

In [ ]:
from os.path import isdir

if not isdir(
    location_to_dataset
):
    !git clone https://github.com/TargetSpectre/Location-Identification-DataSet.git {name_of_location} 
    !mv ./{name_of_location}/DataSet {location_to_dataset}

!ls {location_to_dataset}

# 

In [ ]:
from os import listdir
from os.path import join

labels: list = list()
found_dirs = listdir(
    location_to_dataset
)

for idx in range(
    len(
        found_dirs
    )
):
    selected = found_dirs[idx]
    labels.append(
        selected
    )

    found_dirs[idx] = join(
        location_to_dataset, 
        selected
    )

labels.sort()

print(
    labels
)

In [ ]:
from random import SystemRandom
from keras.utils import image_dataset_from_directory

In [ ]:
batch_size: int = 2
buffer_size: int = 10

number_of_training_iterations: int = 5  

vision_width: int = 960
vision_height: int = 540

maximum_value: int = 65535

In [ ]:
from keras import layers
from keras.models import Model, load_model
from keras.optimizers import Adam

In [ ]:
model = load_model(
    location_to_saved_model
)

model.summary()

model.compile(
    optimizer   = Adam(
        learning_rate   = 0.001,
        amsgrad         = True,
        global_clipnorm = 10.0,
        use_ema         = True
    ),
    loss        = 'categorical_crossentropy',
    metrics     = ['accuracy']
)

In [ ]:
from keras.callbacks \
    import EarlyStopping

callbacks: list = [
    EarlyStopping(
        monitor                 = 'accuracy',
        min_delta               = 0,
        patience                = 0,
        mode                    = 'max',
        restore_best_weights    = True,
        start_from_epoch        = 2
    )
]


In [ ]:
def generate_random_int16_number() -> int:
    global maximum_value
    return SystemRandom().randint(
            1, 
            maximum_value
    )

In [ ]:
def generate_training_dataset():
    global batch_size, vision_height, vision_width
    
    return image_dataset_from_directory(
        location_to_dataset,
        labels                  = 'inferred',
        label_mode              = 'categorical',
        color_mode              = 'rgb',
        batch_size              = batch_size,
        image_size              = (
            vision_height, 
            vision_width
        ),
        shuffle                 = True,
        crop_to_aspect_ratio    = True,
        subset                  = 'training',
        validation_split        = 0.25,
        seed                    = generate_random_int16_number()
    )

In [ ]:
def generate_validation_dataset():
    global batch_size, vision_height, vision_width
    return image_dataset_from_directory(
        location_to_dataset,
        labels                  = 'inferred',
        label_mode              = 'categorical',
        color_mode              = 'rgb',
        batch_size              = batch_size,
        image_size              = (
            vision_height, 
            vision_width
        ),
        shuffle                 = True,
        crop_to_aspect_ratio    = True,
        subset                  = 'validation',
        validation_split        = 0.25,
        seed                    = generate_random_int16_number()
    )

In [ ]:
number_of_epochs: int = 10

In [ ]:
def train():
    global callbacks, number_of_epochs
    
    training_dataset = generate_training_dataset()
    validation_dataset = generate_validation_dataset()

    history = model.fit(
        training_dataset.prefetch(
            buffer_size = buffer_size
        ), 
        epochs              = number_of_epochs, 
        validation_data     = validation_dataset.prefetch(
                                buffer_size = buffer_size
                              ),
        workers             = 12,
        callbacks           = callbacks,
        verbose             = 2
    )

In [ ]:
for idx in range(
    number_of_training_iterations
):
    train()

model.save(
    location_to_saved_model,
    overwrite = True
)